# Actividad 2.1. Cifrado César, sustitución monoalfabética y Vigenère

- Juan Pablo Echeagaray González
- A00830646
- Análisis de Criptografía y Seguridad
- Profesores:
  - Dr. Alberto F. Martínez
  - Dr.-Ing. Jonathan Montalvo-Urquizo
- 24 de mayo del 2022

## Dependencias

In [ ]:
from string import ascii_letters, digits
from random import sample


## Cifrado César

### Encriptado César

In [ ]:
def encrypt_caesar(message: str, offset: int):

    result = ''
    for char in message:
        # Check if its uppercase
        if char.isupper():
            result += chr((ord(char) + offset - 65) % 26 + 65)
        else:
            result += chr((ord(char) + offset - 97) % 26 + 97)

    return result


### Rompiendo cifrado César

In [ ]:
def break_caesar(message: str, known_key: int = None):
    alphabet = ascii_letters[len(ascii_letters) // 2:]
    res = []

    if known_key is None:
        search_space = range(len(alphabet))
    else:
        search_space = [known_key]
    
    for key in search_space:
        translated = ''
        for char in message:
            if char in alphabet:
                num = alphabet.find(char)
                num -= key
                if num < 0:
                    num += len(alphabet)
                translated += alphabet[num]
            else:
                translated += char
        
        res.append([key, translated])
        # Map list of lists to dict
        out = dict(res)

    return out


### Probando cifrado y desencriptado César

In [ ]:
def caesar_encryption(plain_text: str, mode: str):

    shift = 3
    ciphert_text = encrypt_caesar(plain_text, shift)
    if mode == 'known_key':
        result_2 = break_caesar(ciphert_text, shift)
        return f'''
        Plain text: {plain_text}
        Shift: {shift}
        Cipher text: {ciphert_text}
        Known-key
        Known-key-result: {result_2}'''

    elif mode == 'brute-force':
        result_1 = break_caesar(ciphert_text)
        return f'''
        Plain text: {plain_text}
        Shift: {shift}
        Cipher text: {ciphert_text}
        Brute Force
        Result: {result_1}'''
        
    else:
        print('Invalid mode')
    

In [ ]:
plain_text = 'PERO MIRA COMO BEBEN LOS PECES EN EL RIO'

In [ ]:
%%timeit -n 5000
caesar_encryption(plain_text, 'known_key')


In [ ]:
%%timeit -n 5000
caesar_encryption(plain_text, 'brute-force')


## Cifrado monoalfabético

### Generación de alfabeto aleatorio

In [ ]:
def random_alphabet_table():
    character_pool = ascii_letters + digits + '\n":;.,-+*/'
    orig = list(character_pool)
    shuffled = sample(orig, len(orig))
    key = dict(zip(orig, shuffled))

    return key


### Encriptado

In [ ]:
def encrypt_message(message: str, key: dict):
    encrypted = []
    for char in message:
        if char in key:
            encrypted += key[char]
        else:
            encrypted += char

    return ''.join(encrypted)


### Inverso Alfabeto

In [ ]:
def inv_alphabet(key: dict):

    return {v: k for k, v in key.items()}


### A desencriptar

In [ ]:
def decrypt_message(message: str, key: dict):

    return encrypt_message(message, inv_alphabet(key))
    

### Prueba de monoencriptado

In [ ]:
def mono_encryption():

    file_path = '../../homeworks/ciphers/text2.txt'

    with open(file_path, 'r') as f:
        message = f.readlines()

    message = ''.join(message)

    # Encryption
    cipher = random_alphabet_table()
    encrypted = encrypt_message(message, cipher)
    decrypted = decrypt_message(encrypted, cipher)

    return decrypted


In [ ]:
mono_encryption()


In [33]:
def mono_frequency_analysis(cipher_text: str, language: str):
    
    frequencies = {'eng': 'ETAOINSHRDLCUMWFGYPBVKJXQZ', 
                    'spa': 'EAOSNRILDTUCMPBHQYVGFJZXKW', 
                    'fra': 'EASTIRNULODMCPVHGFBQJXZYKW'}

    if language not in frequencies:
        print('Invalid language')
        return
    
    lang_frequencies = frequencies[language]


mono_frequency_analysis('', 'eng')

## Cifrado Vigenère

In [ ]:
def transform_plain(plain_text: str, key_word: str) -> str:
    """Transform plain text with a keyword using a substitution

    Args:
        plain_text (str): Plain text to be transformed
        key_word (str): Word to map plain text with

    Returns:
        str: Mapped text
    """    

    key = list(key_word)

    if len(plain_text) == len(key):
        return key

    else:
        for i in range(len(plain_text) - len(key)):
            key.append(key[i % len(key)])

    mapped_text = ''.join(key)

    return mapped_text


In [ ]:
def encrypt_vigenere(plain_text: str, key: str) -> str:

    alphabet = ascii_letters[len(ascii_letters) // 2:]
    cipher_text = []
    # Cannot use index based for loop, need to handle whitespaces
    key_index = 0
    key = key.upper()

    for char in plain_text:
        
        id = alphabet.find(char.upper())

        # Check if its not contained in the alphabet
        if id != -1:
            pass
        else:
            cipher_text.append(char)

    return ''.join(cipher_text)


In [ ]:
text = 'HOLAMUNDO'
key = 'PERRO'
key_word = transform_plain(text, key)
cipher_text = encrypt_vigenere(text, key_word)
print(f'''
Text: {text}
Key: {key}
Key word: {key_word}
Cipher text: {cipher_text}''')

In [ ]:
def decrypt_vigenere(cipher_text: str, key_word: str) -> str:
    plain_text = []
    for i in range(len(plain_text)):
        
        if cipher_text[i].isalpha():
            x = (ord(cipher_text[i]) - ord(key_word[i] % len(key_word))) % 26
            x += ord('A')
            plain_text.append(chr(x))
            
        else:
            plain_text.append(cipher_text[i])
        
    return ''.join(plain_text)
    

In [ ]:
def vigenere_test():
    text = 'HOLAMUNDO'
    key = 'MONA'
    key_word = keyword_generator(text, key)
    encrypted = encrypt_vigenere(text, key_word)
    decrypted = decrypt_vigenere(encrypted, key_word)

    print(f'''Vigenere Cipher
    Text: {text}
    Key: {key_word}
    Encrypted: {encrypted}
    Decrypted: {decrypted}''')


vigenere_test()
